In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [16]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *


from PIL import Image
from numpy import array

import skimage
import time

import cv2
import numpy as np

from torchvision.utils import save_image

In [1]:
# convert to one-hot
# 32 x image
# for each pixel, have 1 value that is 1 (corresponding to that pixel's channel/class), and 31 values that are 0
frame_1hot = ...

In [3]:
cap = cv2.VideoCapture("/home/jupyter/.fastai/data/camvid/opticalFlow/train.mp4")

In [5]:
ret, frame1 = cap.read()

In [9]:
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY) 
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

In [10]:
type(prvs)

numpy.ndarray

In [23]:
def visualizeIT(ang, mag, hsv, VISUALIZE_FLOW):
        if VISUALIZE_FLOW:
            hsv[...,0] = ang*180/np.pi/2
            hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
            rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

            cv2.imshow('frame2',rgb)
            k = cv2.waitKey(30) & 0xff
        if k == 27:
            pass
        elif k == ord('s'):
            cv2.imwrite('opticalfb.png',frame2)
            cv2.imwrite('opticalhsv.png',rgb)

### Test on <10 frames

In [24]:
ret, frame2 = cap.read()
next = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

flow = cv2.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

In [28]:
magAng = np.stack([mag, ang])
type(magAng)

numpy.ndarray

#### Save to an image
Check to see if image works

In [29]:
imgTest = cv2.imwrite('optFlow67.png', magAng)

In [30]:
imgTest

False

## Try on ALL frames

In [20]:
# True == visualize the motion
VISUALIZE_FLOW = False
count = 1

while(1):
    # compute optical flow for each frame
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Save mag, ang as information relevant to motion at each pixel 
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])

    # Combine mag, ang into one variable 
    magang = np.stack([mag, ang]) # frame2.shape x 2
    
    
    cv2.imwrite(f'/home/jupyter/.fastai/data/camvid/opticalFlow/optFlow{count}.png', magang)
    
#     save_image(magang, f'optFlow{count}.png')
    # save out frame2 motion
#     save_image(mag, ...) # how much motion?
#     save_image(ang, ...) # motion in what direction?


    # Set prvs to next image
    prvs = next
    count += 1
    
    # visualize the motion of frame2
    visualizeIT(ang, mag, hsv, VISUALIZE_FLOW)

KeyboardInterrupt: 

### Pooling and One-Hot Encoding

In [ ]:
for frame in frames:
    # load frame_1hot for frame
    ...
    # load magang for frame
    ...
  
    # combine flow and segmentation information
    features = np.dot(frame_1hot, magang) # 32x2ximage
    
    # pool to downsample
    # use mean pooling over 3x3 grid on the image dimensions
    # 32x2x(3x3) vector of length 576
    features = pool(features)
    
    vector = np.ravel(features)
    

cap.release()
cv2.destroyAllWindows()